### Fine-tuning a language model

**Data generation using an LLM**: Uses a Large model like Llama-3 (70B) to generate data to use for fine-tuning a small model like Phi 3 (3.8B).

In [ ]:
%pip install pandas groq
%pip install "unsloth @ git+https://github.com/unslothai/unsloth.git"
%pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

### Generate synthetic data for fine-tuning

In [12]:
import os
import re
import pandas as pd
from transformers import TextStreamer
from groq import Groq

data_filename = os.path.join("datasets", "customer_support_bot_finetune_data.csv")

# if file exists, read it
if os.path.exists(data_filename):
	with open(data_filename, "r") as file:
		csv_text = file.read()
else:
	client = Groq(
		api_key=os.environ.get("GROQ_API_KEY"),
	)

	lines = 100
	prompt = "Generate high-quality data for fine-tuning in csv for customer support chatbot" \
			f" for an ecommerce platform in {lines} lines of data. fields: instruction, output." \
			"Include the csv file text in triple quotes ```. " \
			"response should include no other text."
	chat_completion = client.chat.completions.create(
		messages=[{ "role": "user", "content": prompt }],
		model="llama3-70b-8192",
	)

	response = chat_completion.choices[0].message.content
	if not response:
		raise SystemExit("No response from the API.")

	# if response doesnt end with ``` then add it
	if not response.endswith("```"):
		response += "```"

	# get the data from the response - json object between triple quotes ``` ```
	match = re.search(r'```(.*?)```', response, re.DOTALL)
	if match:
		csv_text = match.group(1)
		csv_text = csv_text.strip()
		# write to json file
		with open(data_filename, "w") as file:
			file.write(csv_text)
	else:
		print(response)
		raise SystemExit("No data found in the response.")

# parse the csv text
df = pd.read_csv(data_filename)
training_data = df.to_dict(orient="records")

print(f"Data size: {len(training_data)}")
training_data[0]

Data size: 56


{'instruction': 'What is the status of my order?',
 'output': 'Your order is currently being processed. Please allow 3-5 business days for shipping.'}

### Prepare the model for fine-tuning

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
	model_name = "unsloth/Phi-3-mini-4k-instruct",
	max_seq_length = max_seq_length,
	dtype = None,  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
	load_in_4bit = True, # Use 4bit quantization to reduce memory usage. Can be False.
)

# model = FastLanguageModel.get_peft_model(
# 	model,
# 	r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
# 	target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
# 						"gate_proj", "up_proj", "down_proj",],
# 	lora_alpha = 16,
# 	lora_dropout = 0, # Supports any, but = 0 is optimized
# 	bias = "none",    # Supports any, but = "none" is optimized
# 	# [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
# 	use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
# 	random_state = 3407,
# 	use_rslora = False,  # We support rank stabilized LoRA
# 	loftq_config = None, # And LoftQ
# )

prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(data_row):
    instructions = data_row["instruction"]
    outputs = data_row["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        # without EOS_TOKEN, generation will go on forever
        text = prompt.format(instruction, output) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

dataset = training_data.map(formatting_prompts_func, batched = True,)

### Train the model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

# # Save the model and tokenizer
# model.save_pretrained("customer_support_model")
# tokenizer.save_pretrained("customer_support_model")

### Test the model

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        "May I know the order status?",
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)